# Julia code for ACE basis evaluation

## Misc

In [1]:
include("misc.jl")

parse_dict (generic function with 1 method)

In [2]:
# full path not necessary if .json file is in the same dir
string_dict = JSON.parsefile("evACE.json")
symbol_dict = parse_dict("evACE.json")

Dict{Symbol, Dict{Symbol}} with 3 entries:
  :evaluation => Dict(:c_atom=>"Li")
  :trajectory => Dict(:index=>"0:100:1", :traj_file=>"./examples/traj_2.1_0-100…
  :ace_basis  => Dict{Symbol, Any}(:species=>Any["H", "C", "O", "F", "P", "Li",…

In [3]:
symbol_dict[:trajectory]

Dict{Symbol, String} with 2 entries:
  :index     => "0:100:1"
  :traj_file => "./examples/traj_2.1_0-100-1_newatoms.xyz"

## Gen ACE basis

In [4]:
include("descriptor.jl")

┌ Info: Precompiling ACE1pack [8c4e8d19-0bd6-4234-8309-7210652e3178]
└ @ Base loading.jl:1423
┌ Info: Skipping precompilation since __precompile__(false). Importing ACE1pack [8c4e8d19-0bd6-4234-8309-7210652e3178].
└ @ Base loading.jl:1124
┌ Info: Precompiling IPFitting [3002bd4c-79e4-52ce-b924-91256dde4e52]
└ @ Base loading.jl:1423
┌ Info: Skipping precompilation since __precompile__(false). Importing IPFitting [3002bd4c-79e4-52ce-b924-91256dde4e52].
└ @ Base loading.jl:1124
┌ Info: Precompiling ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c]
└ @ Base loading.jl:1423
┌ Info: Skipping precompilation since __precompile__(false). Importing ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c].
└ @ Base loading.jl:1124


ACEdescriptor (generic function with 1 method)

In [5]:
ace_basis = dict_to_ace_basis(symbol_dict)
println(length(ace_basis))

5120


In [6]:
# input to generate the ACE basis
# -> some parameters are place by default and other are hidden
symbol_dict[:ace_basis]

Dict{Symbol, Any} with 7 entries:
  :species => ["H", "C", "O", "F", "P", "Li", "V", "Sc"]
  :N       => 3
  :maxdeg  => 4
  :rin     => 0.1
  :rcut    => 5.5
  :pin     => 2
  :r0      => 0.5

## Load the trajectory

In [7]:
include("trajectory.jl")

_traj_init (generic function with 1 method)

In [8]:
symbol_dict[:trajectory]

Dict{Symbol, String} with 2 entries:
  :index     => "0:100:1"
  :traj_file => "./examples/traj_2.1_0-100-1_newatoms.xyz"

In [9]:
# read the trajectory
traj_db = read_xyz(; symbol_dict[:trajectory]...)
println(length(traj_db))

┌ Info: Keys used: E => "", F => "", V => ""
└ @ IPFitting.Data /home/agardin/.julia/packages/IPFitting/E09UD/src/data.jl:152
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


┌─────────────┬───────┬────────┬───────┬─────────┬───────┐
│ config_type │ #cfgs │  #envs │    #E │      #F │    #V │
│      String │ Int64 │  Int64 │ Int64 │   Int64 │ Int64 │
├─────────────┼───────┼────────┼───────┼─────────┼───────┤
│     nothing │   100 │ 840200 │     0 │       0 │     0 │
├─────────────┼───────┼────────┼───────┼─────────┼───────┤
│       total │   100 │ 840200 │     0 │       0 │     0 │
│     missing │     0 │      0 │   100 │ 2520600 │   900 │
└─────────────┴───────┴────────┴───────┴─────────┴───────┘
100


### ACE evaluation

In [13]:
central_Z = findall(x->x==symbol_dict[:evaluation][:c_atom], 
                    elements_dict["symbols"]
                    )

1-element Vector{Int64}:
 3

In [14]:
eval_sites = findall(x->x==central_Z[1], traj_db[1].at.Z)
println(length(eval_sites))

114


In [15]:
single_conf = [ACE1pack.JuLIP.site_energy(ace_basis, traj_db[1].at, site) for site in eval_sites]
println(length(single_conf), " ", length(single_conf[1]))

114 5120


In [17]:
single_site = ACE1pack.JuLIP.site_energy(ace_basis, traj_db[1].at, eval_sites[1])
println(length(single_site))

5120


In [18]:
typeof(single_conf)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [19]:
typeof(single_site)

Vector{Float64} (alias for Array{Float64, 1})

In [31]:
function ACEdescriptor(; traj_db, basis, c_atom)
    # express central atom as Z
    central_Z = findall(x->x==c_atom, 
                elements_dict["symbols"])[1]
    # get the indxs of the sites to be evaluated
    eval_sites = findall(x->x==central_Z, traj_db[1].at.Z)

    return [[ACE1pack.JuLIP.site_energy(ace_basis, at_conf.at, site) 
            for site in eval_sites] 
            for at_conf in traj_db]
end

ACEdescriptor (generic function with 2 methods)

In [33]:
ace_descr = ACEdescriptor(traj_db = traj_db, 
                          basis = ace_basis, 
                          c_atom = symbol_dict[:evaluation][:c_atom])
size(ace_descr)

(100,)

In [37]:
using HDF5

In [40]:
h5write("ace_descr_test.h5", "./.", ace_descr)

LoadError: MethodError: no method matching datatype(::Vector{Vector{Vector{Float64}}})
[0mClosest candidates are:
[0m  datatype([91m::HDF5.VLen{T}[39m) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.Reference} at ~/.julia/packages/HDF5/7zvRl/src/typeconversions.jl:61
[0m  datatype([91m::HDF5.VLen{C}[39m) where C<:HDF5.CharType at ~/.julia/packages/HDF5/7zvRl/src/typeconversions.jl:62
[0m  datatype([91m::Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.Reference}[39m) at ~/.julia/packages/HDF5/7zvRl/src/typeconversions.jl:167
[0m  ...

In [26]:
atZ = [convert(Int, Z) for Z in unique(sort(traj_init[1].at.Z))]

8-element Vector{Int64}:
  1
  3
  6
  8
  9
 15
 21
 23